# Sommaire

# importation de la base et préprocessing

In [1]:
import pandas as pd
import numpy as np 
# import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import  RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor

from sklearn.dummy import DummyRegressor
import xgboost as xgb
import multiprocessing
from xgboost import XGBRegressor
from category_encoders import TargetEncoder
import category_encoders as ce
from category_encoders import QuantileEncoder
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from julearn import run_cross_validation
from julearn.utils import configure_logging
from julearn.model_selection import StratifiedGroupsKFold
from julearn.scoring import register_scorer
import math
pd.set_option('display.max_rows', 25)

In [2]:
buildingnr= pd.read_csv("buildingnr.csv",decimal=".",index_col=0)


In [3]:
#création d'une classe pour segmenter le quantile encoder 
buildingnr['log_SiteEnergyUse(kBtu)_class']= pd.qcut(buildingnr['log_SiteEnergyUse(kBtu)'],4,labels=False)


In [4]:
# SFE 50 feature + YearsENERGYSTARCertified ENERGYSTARScore 
base_ml = buildingnr[['log_SiteEnergyUse(kBtu)_class','log_SiteEnergyUse(kBtu)',
 #                    'YearsENERGYSTARCertified', 'ENERGYSTARScore',
    'Area',
 'NaturalGas%',
 'LPU2%',
 'LPU3%',
 'log_PropertyGFATotal',
 'Supermarket/Grocery Store_GFA',
 'Data Center_GFA',
 'Restaurant_GFA',
 'Distribution Center_GFA',
 'Office_GFA%',
 'Office_eui%',
 'Non-Refrigerated Warehouse_eui%',
 'BuildingType',
 'Retail Store_GFA',
 'Non-Refrigerated Warehouse_GFA',
 'Self-Storage Facility_GFA',
 'Office_GFA',
 'Other - Recreation_GFA%',
 'Parking_eui%',
 'Other_GFA',
 'Supermarket/Grocery Store_GFA%',
 'Restaurant_GFA%',
 'Distribution Center_GFA%',
 'Other_GFA%',
 'Retail Store_GFA%',
 'LPU1%',
 'Non-Refrigerated Warehouse_GFA%',
 'Electricity%',
 'ThirdLargestPropertyUseType',
 'PrimaryPropertyType',
 'log_NaturalGas%',
 'log_Electricity%',
 'Latitude',
 'Longitude',
 'NumberofFloors',
 'PropertyGFATotal',
 'PropertyGFABuilding(s)',
 'LargestPropertyUseType',
 'LargestPropertyUseTypeGFA',
 'SecondLargestPropertyUseType',
 'SecondLargestPropertyUseTypeGFA',
 'log_parking%',
 'log_SecondLargestPropertyUseTypeGFA',
 'log_LargestPropertyUseTypeGFA',
 'log_ThirdLargestPropertyUseTypeGFA',
 'log_PropertyGFABuilding(s)',
 'DecBuilt',
 'parking%',
 'LargestUse']]


In [5]:
base_ml.shape
Report = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
#initialisation tableau de reporting total 
Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée'])
#initialisation tableau de reporting pour un fold


In [6]:
colonnes = base_ml.select_dtypes(include= 'number').columns.tolist()
for col in colonnes : 
    mask = base_ml[col].isna()
    base_ml.loc[mask,col]= 0
    
colonnes = base_ml.select_dtypes(include= 'object').columns.tolist()
for col in colonnes : 
    mask = base_ml[col].isna()
    base_ml.loc[mask,col]= 'missing'
    
X = base_ml.drop(columns = ['log_SiteEnergyUse(kBtu)_class','log_SiteEnergyUse(kBtu)'])
y = base_ml['log_SiteEnergyUse(kBtu)']


colQE= base_ml.select_dtypes(include= 'object').columns.tolist()
colQE.append('DecBuilt')

enc = QuantileEncoder(cols=colQE, quantile=0.5, m=1.0).fit(X, y)
X_QE= enc.transform(X)

X=X_QE

X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Area,1634.0,14.842038,0.414714,14.328011,14.384285,14.767478,15.280563,15.470416
NaturalGas%,1634.0,0.279909,0.263912,0.000000,0.000000,0.246140,0.494383,1.000000
LPU2%,1634.0,0.119654,0.154132,0.000000,0.000000,0.000000,0.243169,0.500000
LPU3%,1634.0,0.021988,0.057969,0.000000,0.000000,0.000000,0.000000,0.333333
log_PropertyGFATotal,1634.0,11.047849,0.959378,9.331230,10.288894,10.804583,11.569063,16.047690
...,...,...,...,...,...,...,...,...
log_ThirdLargestPropertyUseTypeGFA,1634.0,1.761119,3.566186,0.000000,0.000000,0.000000,0.000000,13.038434
log_PropertyGFABuilding(s),1634.0,10.967065,0.914419,8.198639,10.254285,10.762700,11.460958,16.047690
DecBuilt,1634.0,14.856088,0.442785,14.282018,14.557876,14.720899,14.868327,15.635333
parking%,1634.0,0.062093,0.141092,0.000000,0.000000,0.000000,0.000000,0.895023


In [7]:
nsplit = 5 # nombre de split du StratifiedKfold

# RandomForest

In [8]:
#Random Forest 
debut = time.time()

estimator = RandomForestRegressor(random_state=22, n_estimators = 2000)

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', estimator) ])


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)
      
    
Append= pd.DataFrame([['RandomForest' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    

Report = pd.concat([Report, Append]) 
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382


# Linear Regression

In [9]:
### LinearRegression
debut = time.time()
Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée'])


reg = Pipeline(steps=[   ('preprocessor', numeric_transformer), ('regression', linear_model.LinearRegression())])
  


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

reg.fit(x_train, y_train)
    
Append = pd.DataFrame([['LinearRegression' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    


Report = pd.concat([Report, Append]) 
Report


,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131


# Dummy 

In [10]:
debut = time.time()
#Dummy regressor 

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', DummyRegressor())])


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Dummy' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868


# Ridge

In [11]:
#Ridge
debut = time.time()
reg = Pipeline(steps=[('preprocessor',numeric_transformer),
                      ('regression', linear_model.Ridge(random_state=22 ))])


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Ridge' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419


# RidgeCV

In [12]:
#RidgeCV

debut = time.time()
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', linear_model.RidgeCV())])


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['RidgeCV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794


# Lasso-GSCV

In [13]:
debut = time.time()
parameters = {'alpha': [1, 10, 100,1000,10000], 'tol': [1,0.1,0.01,0.001,0.0001, 0.00001, 0.000001],'selection':['random','cyclic']}
reg = Pipeline(steps=[('preprocessor',numeric_transformer),
                      ('regression', GridSearchCV(linear_model.Lasso(max_iter=10000, random_state=22),parameters))]) 


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Lasso + GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 
Report


,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256


In [14]:
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256


# RandomForest GSCV

In [15]:
#Random Forest CV 
debut = time.time()

tuned_parameters = { 'max_depth': [None, 1, 2, 3], 'min_samples_split': [2,5,10, 50, 100]}
#test sur 5000 estimators max voir temps 750 sec quasi pas d'amélioration) 
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(RandomForestRegressor(random_state=22), tuned_parameters, cv=5, n_jobs=-4, verbose=1))]) #possible de rajouter l'indicateur de scoring : scoring=r2_score par ex. 

# Create KFold object.

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['RandomForest CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    
Report = pd.concat([Report, Append]) 
Report

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits


,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256
0,RandomForest CV,0.385,0.621,0.777,0.03,0.908,231.183138


# XGBoost GSCV

In [16]:
debut = time.time()


parameters = {'n_estimators': [500,1000], #[8000,10000] - R2 0.365
    'max_depth': [2,4],
    'gamma':[0.1,0.2], # Minimum loss reduction to create new tree split ,0.5,0.9
    'subsample':[0.7], 
    'reg_lambda':[0.1], 
    'reg_alpha':[0.1,0.8],
    'n_jobs':[-1],
     'booster':['gbtree']
              }
              
    

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(xgb.XGBRegressor(learning_rate=0.01,objective='reg:squarederror', 
                        random_state = 22) , parameters,scoring = 'r2',cv=5,verbose =1))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['XGB GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 
Report

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256
0,RandomForest CV,0.385,0.621,0.777,0.03,0.908,231.183138
0,XGB GS_CV,0.415,0.644,0.76,0.031,0.835,867.591294


# Elasticnet GSCV

In [ ]:
#ElasticNet 
debut = time.time()

parameters = {'alpha': [1, 10, 100,1000,10000], 'l1_ratio':[0,0.25,0.75,1,100],'tol': [1,0.1,0.01,0.001,0.0001, 0.00001, 0.000001],
              'selection':['random','cyclic']}


reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(ElasticNet(random_state=22,max_iter=1000),parameters))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['ElasticNet GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [ ]:
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256
0,RandomForest CV,0.385,0.621,0.777,0.03,0.908,231.183138
0,XGB GS_CV,0.415,0.644,0.76,0.031,0.835,867.591294
0,ElasticNet GS_CV,0.469,0.685,0.729,0.036,0.667,149.235487


# ElasticNetCV

In [ ]:
### ElasticNet Cross Validation
debut = time.time()
reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', linear_model.ElasticNetCV(random_state=22))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['ElasticCV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [ ]:
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256
0,RandomForest CV,0.385,0.621,0.777,0.03,0.908,231.183138
0,XGB GS_CV,0.415,0.644,0.76,0.031,0.835,867.591294
0,ElasticNet GS_CV,0.469,0.685,0.729,0.036,0.667,149.235487
0,ElasticCV,0.365,0.604,0.789,0.03,0.72,1.427466


# Bagging

In [ ]:
debut = time.time()

reg = Pipeline(steps=[('preprocessor',numeric_transformer),                    
                      ('regression', BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=22))])

skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['BaggingRegressor' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [ ]:
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256
0,RandomForest CV,0.385,0.621,0.777,0.03,0.908,231.183138
0,XGB GS_CV,0.415,0.644,0.76,0.031,0.835,867.591294
0,ElasticNet GS_CV,0.469,0.685,0.729,0.036,0.667,149.235487
0,ElasticCV,0.365,0.604,0.789,0.03,0.72,1.427466


# LinearSVR GSCV

In [ ]:
debut = time.time()
parameters = {'epsilon': [0], 'tol': [0.0001],'C':[1],'loss':["epsilon_insensitive","squared_epsilon_insensitive"],'intercept_scaling':[1],
              'dual':[True, False],'verbose':[0],'random_state':[22],'max_iter':[1000]}

reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                      ('regression', GridSearchCV(LinearSVR(),parameters))]) 


skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 ) 


for train_index, test_index in skf.split(X, base_ml['log_SiteEnergyUse(kBtu)_class']):
    x_train, x_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(x_train, y_train)

        
    
Append= pd.DataFrame([['Linear SVR GS_CV' , 
                        np.around (mean_squared_error(y_test,reg.predict(x_test)),3),
                       np.around (mean_squared_error(y_test,reg.predict(x_test),squared=False),3),
                      np.around(r2_score(y_test,reg.predict(x_test)),3),
                     np.around( mean_absolute_percentage_error(y_test,reg.predict(x_test)),3),
                      np.around(r2_score(y_train,reg.predict(x_train)),3) ,
                      time.time() - debut]], 
                     columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
            
Report = pd.concat([Report, Append]) 


In [ ]:
Report

,méthode,MSE,RMSE,R2 Test,MA%E,R2 Train,Durée
0,RandomForest,0.375,0.612,0.783,0.03,0.952,156.625382
0,LinearRegression,0.365,0.604,0.789,0.031,0.733,0.034131
0,Dummy,1.728,1.315,-0.0,0.071,0.0,0.037868
0,Ridge,0.362,0.602,0.79,0.031,0.733,0.0419
0,RidgeCV,0.359,0.599,0.792,0.031,0.73,0.088794
0,Lasso + GS_CV,1.728,1.315,-0.0,0.071,0.0,4.728256
0,RandomForest CV,0.385,0.621,0.777,0.03,0.908,231.183138
0,XGB GS_CV,0.415,0.644,0.76,0.031,0.835,867.591294
0,ElasticNet GS_CV,0.469,0.685,0.729,0.036,0.667,149.235487
0,ElasticCV,0.365,0.604,0.789,0.03,0.72,1.427466


In [ ]:
#nous retenons RandomForest qui a quasiment le même R2 et MAPE qus XGB avec EnergyStar en étant beaucoup plus rapide